In [1]:
import os
import re
import time
from pathlib import Path
import shutil
import glob
import string
import unicodedata
from datetime import datetime, timedelta

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from sqlalchemy import create_engine
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
#os.path.join(os.getcwd(), 'news')

#for file in glob.iglob('news/**/*.txt', recursive=True):
#    print(file)
#

In [3]:
#str(datetime.now())

In [4]:
#if not os.path.exists('news'):
#    os.makedirs('news')

In [5]:
#os.path.join('news', 'utils')

In [6]:
#datetime.now().strftime('%d%m%Y')

In [7]:
#os.getcwd()

In [8]:
class ComercioCovidCrawler(object):
    def __init__(self, url_comercio='https://elcomercio.pe/noticias/coronavirus/'):
        #-------------------------- Habilitacion de ruta --------------------------#
        self.folder_news = 'news'
        if not os.path.exists(self.folder_news):
            os.makedirs(self.folder_news)
            
        self.hoy = datetime.today()
        self.hoy_format = self.hoy.strftime('%d%m%Y')
        self.newsletter_path = os.path.join(self.folder_news, self.hoy_format, 'original')
        if not os.path.exists(self.newsletter_path):
            os.makedirs(self.newsletter_path)
        
        self.discarded_path = os.path.join(self.folder_news, self.hoy_format, 'descartado')
        if not os.path.exists(self.discarded_path):
            os.makedirs(self.discarded_path)
            
        self.flg_selected = True
        #--------------------------------------------------------------------------#
        
        self.current_path = os.path.join(os.getcwd(), 'utils')
        self.links = []
        self.news = []
        self.discarded = []
        self.tags_descartados = ['/tvmas/', '/mundo/', '/videos/']
        
        options = Options()
        options.add_argument("--incognito")
        options.add_experimental_option("prefs", {
            "download.default_directory": r"{0}".format(self.current_path),
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "safebrowsing.enabled": True
        })
        
        self.driver_path = os.path.join(self.current_path, 'chromedriver.exe')
        print(self.driver_path )
        self.url = url_comercio
        self.driver = webdriver.Chrome(self.driver_path, options=options)
    
    def page_load(self, url=''):
        if url == '':
            url = self.url
            
        self.driver.get(url)
        print(" ")
        print("Ingresando: {}".format(url))
        time.sleep(3)
        
    def search_links(self):
        elements = self.driver.find_elements_by_class_name("story-item__information-box")
        self.links = [element.find_element_by_tag_name('a').get_attribute('href') for element in elements]
        print("{} noticias encontradas".format(len(self.links)))
    
    def descarte(self, new):
        for tag in self.tags_descartados:
            if tag in new:
                return 1
        return 0
    
    def path_related_with_new(self, new):
        name_file = new.split('/')[-2].lower()[:108] + '.txt'
        
        return os.path.join(
            self.newsletter_path if self.flg_selected else self.discarded_path, 
            name_file
        ) 
    
    def validation_exists_in_storage(self, new):
        path_ = self.path_related_with_new(new)
        
        if os.path.isfile(path_):
            print("NOTICIA YA ALMACENADA EL DÍA DE HOY")
            return None
        
        return path_
    
    def get_content_new(self, new):
        path_ = self.validation_exists_in_storage(new)
        if not path_:
            return None
        
        print(" ")
        print(path_)
        
        ## Load
        self.page_load(new)
        
        ## extract content
        txt_ = '\n'.join(
            [el.text for el in self.driver.find_elements_by_class_name('story-contents__font-paragraph')]
        ).strip()
        if len(txt_) == 0:
            raise Exception("ERROR: CONTENIDO NULO DE NOTICIA POR SOLICITUD DE SUBSCRIPCIÓN")
            
        ## SAving
        myfile = Path(path_)
        myfile.touch(exist_ok=True)
        
        with open(path_, 'w+', encoding="utf-8") as file:
            file.write(txt_)
            
        
    def storage(self, selected=True):
        self.flg_selected = selected
        
        msje = 'Seleccionadas: ' if selected else 'Descartadas'
        listado = self.news if selected else self.discarded
        print(msje)
        print(listado)
        print("-"*50)
        
        for new in listado:
            try:
                self.get_content_new(new)
                print("-"*100)
            except Exception as e:
                print("ERROR: ", str(e))
                print("-"*100)
    
    def get_relevants_news(self):
        seleccionadas = [new for new in self.links if self.descarte(new) == 0]
        descartadas = [new for new in self.links if new not in seleccionadas]
        
        print("{} noticias descartadas".format(len(descartadas)))
        print("-"*50)
        print("{} noticias seleccionadas".format(len(seleccionadas)))
        print("")
        #print(seleccionadas)
        self.news = seleccionadas
        self.discarded = descartadas

In [9]:
#ComercioCovidCrawler()

In [10]:
try:
    comercio.driver.close()
except:
    pass

comercio = ComercioCovidCrawler()
comercio.page_load()
comercio.search_links()
comercio.get_relevants_news()
comercio.storage(selected=True)
comercio.storage(selected=False)


try:
    comercio.driver.close()
except:
    pass

C:\Users\Villacorta\STORAGE\tesis_maestria_ciencia_datos\utils\chromedriver.exe
 
Ingresando: https://elcomercio.pe/noticias/coronavirus/
50 noticias encontradas
25 noticias descartadas
--------------------------------------------------
25 noticias seleccionadas

Seleccionadas: 
['https://elcomercio.pe/lima/sucesos/vacuna-contra-el-covid-19-mas-de-trece-millones-849-mil-peruanos-ya-fueron-inmunizados-contra-el-coronavirus-sinopharm-tercer-ola-coronavirus-nndc-noticia/', 'https://elcomercio.pe/lima/sucesos/covid-19-en-peru-lima-metropolitana-y-callao-siguen-en-nivel-moderado-hasta-el-3-de-octubre-tercera-ola-emergencia-sanitaria-nndc-noticia/', 'https://elcomercio.pe/lima/sucesos/covid-19-vacunafest-estos-son-los-54-establecimientos-de-salud-para-este-fin-de-semana-tercera-ola-coronavirus-minsa-nndc-noticia/', 'https://elcomercio.pe/lima/sucesos/toque-de-queda-en-lima-y-callao-continuara-desde-la-100-am-hasta-las-400-am-estado-de-emergencia-nacional-covid-19-nndc-noticia/', 'https://elc

In [11]:
#comercio.links